Task 1: Multi-layer ANN

Hyperparameters:
 1. Number of nodes
 2. Number of layers
 3. Activation function

In [1]:
# Load library
import pandas as pd
import numpy as np
import math
from enum import Enum
from sklearn.model_selection import train_test_split

In [2]:
# Load data
concrete = pd.read_csv('data/concrete_data.csv')
concrete.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
# Separate X and Y
# Then separate test and train set
# Also do the Cross-Validation (optional)
X = concrete.drop('concrete_compressive_strength', axis = 1)
y = concrete['concrete_compressive_strength']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 42)

In [4]:
# Activation function
def logistic(x):
    return 1/(1 + math.exp(-x))

def ReLU(x):
    return max(0, x)

def hyperbolic(x):
    return math.tanh(x)

class ActFunc(Enum):
    log = logistic
    relu = ReLU
    hb = hyperbolic

In [5]:
# Neural Network test!
from neuralNet import neuralNet
from layer import layer

network = neuralNet()

network.add(layer(ActFunc.relu,3))
#network.add(layer(ActFunc.relu,8))
#network.add(layer(ActFunc.hb, 10))
network.add(layer(ActFunc.relu,1))

Applying NN on the concrete data

In [6]:
# PSO test!
import pso

swarmsize = 10
alpha = 0.7
beta = 2
gamma = 1.5
delta = 1.5
epsilon = 0.4
n_iter = 10

particle_swarm_opti = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter)
opti_particle, best_mae_arr = particle_swarm_opti.optimise()



Current best mae: 1889.543907334575
Current best mae: 1313.925592360385
Current best mae: 414.99954826490443
Current best mae: 295.1783688193276
Current best mae: 92.67556397606454
Current best mae: 36.02132686084143
Current best mae: 23.67017158792931
Current best mae: 23.502309690119812
Final best mae 23.502309690119812


In [7]:
best_mae_arr

[1889.543907334575,
 1313.925592360385,
 414.99954826490443,
 295.1783688193276,
 92.67556397606454,
 36.02132686084143,
 23.67017158792931,
 23.502309690119812]

In [8]:
# Testing the optimisation
# Ooooo bad bad result :D
particle_swarm_opti.assessFitness(opti_particle, y_train)

23.502309690119812

In [ ]:
# Testing with the test set
weights, bias = particle_swarm_opti.assessFitness_helper(opti_particle) # Extract the optimised weight and bias
# Apply on the NN
y_pred = X_test.apply(network.forwardCalculation, args = (weights, bias), axis = 1)
mse = network.errorCalculation(y_pred, y_test)
mse

24.70735525976428

In [10]:
# Potential ranges of PSO parameters to try
# Could try them like a gridsearch but its probably a bit too much for that, tuning one at a time is probably the way to go

alphaRange = np.arange(0.4,0.9,0.1)
betaRange = np.arange(1.5,2.5,0.1)
gammaRange = np.arange(1.5,2.5,0.1)
deltaRange = np.arange(1.5,2.5,0.1)
epsilonRange = np.arange(0.1,0.5,0.1) # not sure about the range of the learning rate, might need to experiment or look for more sources

In [11]:
#alpha tuning

accuracies = []

for alpha in alphaRange:
    network = neuralNet()

In [ ]:
# Try neural network architectures
# 2-10 layers
# 3-10 neurons per layer

layers = range(15,22)

# Try different layers
# Constant 4 neurons per layer, activation func relu

accuraciesForLayers = []
for numLayers in layers:
    network = neuralNet()
    network.add(layer(ActFunc.hb,4))
    for i in range(numLayers - 1):
        network.add(layer(ActFunc.relu,4))
    layerTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = layerTestPSO.optimise()
    accuraciesForLayers.append(best_mae_arr[-1])


In [13]:
accuraciesForLayers

[12.23452522962312,
 14.229330975272156,
 22.732558658876062,
 20.994587077337055,
 36.02132686084143,
 13.109981349250907,
 35.679531136512374]

In [ ]:
neurons = range(3,11)

accuraciesForNeuronCounts = []

for neuronCount in neurons:
    network = neuralNet()
    network.add(layer(ActFunc.hb,neuronCount))
    for i in range(4):
        network.add(layer(ActFunc.relu,neuronCount))
    network.add(layer(ActFunc.relu,1))
    neuronsTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = neuronsTestPSO.optimise()
    accuraciesForNeuronCounts.append(best_mae_arr[-1])

In [15]:
accuraciesForNeuronCounts

[25.702233667393035,
 34.446782426000574,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143]

In [ ]:
accuraciesForAlphas = []

for a in alphaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    alphaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, a, beta, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = alphaTestPSO.optimise()
    accuraciesForAlphas.append(best_mae_arr[-1])

In [ ]:
accuraciesForBetas = []

for b in betaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    betaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, b, gamma, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = betaTestPSO.optimise()
    accuraciesForBetas.append(best_mae_arr[-1])

In [ ]:
accuraciesForGammas = []

for g in gammaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    gammaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, g, delta, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = gammaTestPSO.optimise()
    accuraciesForGammas.append(best_mae_arr[-1])

In [ ]:
accuraciesForDeltas = []

for d in deltaRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    deltaTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, d, epsilon, n_iter,prints=False)
    opti_particle, best_mae_arr = deltaTestPSO.optimise()
    accuraciesForDeltas.append(best_mae_arr[-1])

In [ ]:
accuraciesForEpsilons = []

for e in epsilonRange:
    network = neuralNet()
    network.add(layer(ActFunc.hb,10))
    for i in range(4):
        network.add(layer(ActFunc.relu,10))
    network.add(layer(ActFunc.relu,1))
    epsilonTestPSO = pso.PSO(X_train, y_train, network, swarmsize, alpha, beta, gamma, delta, e, n_iter,prints=False)
    opti_particle, best_mae_arr = epsilonTestPSO.optimise()
    accuraciesForEpsilons.append(best_mae_arr[-1])

In [21]:
accuraciesForAlphas

[15354.377627485665,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 35.299261777222505]

In [22]:
accuraciesForBetas

[36.02132686084143,
 20.474822937362575,
 12.682278082837694,
 36.02132686084143,
 18.207566320165597,
 36.02132686084143,
 36.02132686084143,
 26.210351367669173,
 16.359392565852055,
 35.99258021602731]

In [23]:
accuraciesForGammas

[32.42793848927428,
 30.25124079657996,
 17.987898039722555,
 23.841758789922483,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 36.02132686084143,
 30.466267872983728]

In [24]:
accuraciesForDeltas

[36.02132686084143,
 36.02132686084143,
 29.460691733642765,
 35.81119935893489,
 34.43753161089588,
 36.02132686084143,
 18.592358343219736,
 36.02132686084143,
 35.695044470498054,
 13.798642071620908]

In [25]:
accuraciesForEpsilons

[19463.796435973927, 6033.82312534316, 36.02132686084143, 35.35622055199706]